In [1]:
!pip -q install groq langgraph langchain
!pip -q install langchain-groq

In [2]:
from typing import List, Tuple, Any, Dict
from groq import Groq
from langgraph.graph import Graph
from langchain_groq import ChatGroq

In [3]:
from groq import Groq
import os

GROQ_API_KEY = "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"
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

chat_model = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [28]:
from typing import Dict, Any, List

# Defining the state type
State = Dict[str, Any]

# Defining the node to generate response
def generate_response(state: State) -> State:
    # Append the current user input to the conversation history
    state["conversation_history"].append({"role": "user", "content": state["user_input"]})
    
    # Constructing the messages with the conversation history
    messages = [
        {"role": "system", "content": (
            "You are a chatbot that only discusses video games, gaming consoles, or related topics. If the user's query is unrelated to video games or gaming topics, politely inform them that you can only provide information about video games and related subjects. Do not offer any advice, humor, or additional content that is not directly related to video games."
            "politely refuse to answer that part of the query and remind the user that you can only answer questions about video games."
        )}
    ] + state["conversation_history"]
    
    # Assuming `chat_model` is an instance of a chat model with a `chat.completions.create` method
    response = chat_model.chat.completions.create(
        messages=messages,
        model="llama3-8b-8192"
    )
    
    # Get the response content and appending it to the conversation history
    response_content = response.choices[0].message.content
    state["conversation_history"].append({"role": "assistant", "content": response_content})
    
    # Set the response in the state
    state["response"] = response_content
    return state

# Defining the graph
class Graph:
    def __init__(self):
        self.nodes = {}
        self.entry_point = None
        self.edges = {}

    def add_node(self, name: str, func):
        self.nodes[name] = func

    def set_entry_point(self, name: str):
        self.entry_point = name

    def add_edge(self, from_node: str, to_node: str):
        if from_node not in self.edges:
            self.edges[from_node] = []
        self.edges[from_node].append(to_node)

    def compile(self):
        def run(state: State):
            current_node = self.entry_point
            while current_node:
                state = self.nodes[current_node](state)
                if current_node in self.edges:
                    current_node = self.edges[current_node][0]
                else:
                    current_node = None
            return state
        return run

# Creating and configuring the workflow
workflow = Graph()

workflow.add_node("generate_response", generate_response)

workflow.set_entry_point("generate_response")

# Compiling the graph
app = workflow.compile()

# Initializing the conversation state
initial_state = {
    "user_input": "",
    "conversation_history": []  # Initialize an empty conversation history
}

# making interactive chat loop
def chat_with_llm():
    state = initial_state.copy()
    print("You can start chatting with the video game chatbot. Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "."]:
            print("Ending the chat with me.\nHope you gained some good knowledge from me. Goodbye have a nice day.")
            break
        
        state["user_input"] = user_input
        state = app(state)
        print(f"Chatbot: {state['response']}")

# Start chating with chatbot
chat_with_llm()


You can start chatting with the video game chatbot. Type 'exit' to end the chat.


You:  .


Ending the chat with me.
Hope you gained some good knowledge from me. Goodbye have a nice day.


In [ ]:
# Start chating with chatbot
chat_with_llm()

You can start chatting with the video game chatbot. Type 'exit' to end the chat.


You:  hi, tell me about India


Chatbot: I'd be happy to help, but I'm a chatbot focused on video games and gaming consoles. I can provide information on games or consoles that are set in or relate to India, but I won't be able to provide general information about the country. If you have any specific questions about Indian games or gaming-related topics, I'd be happy to try and assist.


You:  tell me about bgmi


Chatbot: BGMI! PUBG Mobile, also known as BGMI (Battlegrounds Mobile India), is a popular battle royale-style mobile game developed by Tencent Games. It's the Indian version of the widely popular PUBG Mobile game, with some changes made to comply with Indian regulations.

BGMI was launched in India in July 2021, and it has gained a massive following in the country. The game allows players to engage in intense battles with up to 99 other players on various maps, including popular ones like Erangel, Miramar, and Sanhok.

BGMI offers a range of game modes, including solo, duos, and squads, and allows players to customize their characters with a variety of cosmetic items. The game also has a strong focus on community building, with features like chat rooms and live streaming.

BGMI has received a significant update since its initial release, with new features, game modes, and content added regularly. If you're interested in learning more about the game or have specific questions about it, 

You:  ok, tell me how to make biryani while playing the game


Chatbot: I'm afraid I'm a chatbot focused on video games and gaming consoles, and I'm not capable of providing cooking instructions or recipes. However, I can try to provide some general tips on how to balance your gaming and culinary pursuits!

If you're playing BGMI, it's probably not a good idea to try to cook biryani while playing, as it requires your full attention and focus on the game. Instead, I recommend taking regular breaks to play the game, and then dedicating some time to cooking your biryani. You can try to get some inspiration for your biryani recipe from the game's setting or characters, like a virtual Pakistani or Indian market!

Remember, gaming and cooking require different skills and attention, so it's best to focus on one task at a time to ensure you do both justice. If you have any further questions about BGMI or other video games, feel free to ask!


You:  ok got it


Chatbot: I'm glad we could clear that up. Since we're back on topic, are you having fun playing BGMI? Do you have a favorite game mode or map? Let me know if you'd like to discuss more about the game!
